In [1]:
import torch
import numpy as np
from scipy.linalg import fractional_matrix_power
from yaspin import yaspin
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from tqdm.notebook import tqdm
import pandas as pd
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import KFold

In [2]:
k_folds = 5

In [3]:
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')
bert_model.cuda()

SentenceTransformer(
  (0): Transformer(
    (auto_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
       

In [4]:
bert_model.max_seq_length = 1000

In [5]:
torch.manual_seed(10)

In [6]:
embed_size = 768
init_query_embed_size = 768
num_docs = 50
batch_size = 10
epochs = 20

In [7]:
class GCN(torch.nn.Module):
    def __init__(self, n_h, input_features):
        super().__init__()
        W = torch.FloatTensor(input_features, n_h)
        self.weights = torch.nn.Parameter(W)
        self.relu = torch.nn.ReLU()
        stdv = 1. / math.sqrt(self.weights.size(1))
        self.weights.data.uniform_(-stdv, stdv)
        bs = torch.FloatTensor(n_h)
        self.bias = torch.nn.Parameter(bs)
        self.bias.data.uniform_(-stdv, stdv)
    
    def forward(self, nfs, A):
        out = torch.matmul(nfs, self.weights)
        out = torch.matmul(A, out)
        out = out + self.bias
        out = self.relu(out)
        return out

In [37]:
class AmbiguityNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = GCN(10, embed_size)
        self.gcn2 = GCN(10, 10)
        self.gcn3 = GCN(10,10)
        self.dp = torch.nn.Dropout(p = 0.5)
        self.dense1 = torch.nn.Linear(in_features = 500 + init_query_embed_size, out_features = 64)
        self.dense2 = torch.nn.Linear(in_features = 64, out_features = 4)
    
    def forward(self, inputs, A, init_query):
        out = self.gcn1(inputs, A)
        out = self.dp(out)
        out = self.gcn2(out, A)
        out = self.dp(out)
        out = self.gcn3(out, A)
        out = self.dp(out)
        out = torch.flatten(out, 1)
        out = torch.cat((out, init_query), dim = 1)
        out = self.dense1(out)
        out = self.dense2(out)
        return out

In [9]:
requests = pd.read_table('./data/train.tsv', sep = '\t', header = 0).drop_duplicates('topic_id')
requests

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,1,Tell me about Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magazine photo essay ""Barack Oba...",Q00384,are you interested in seeing barack obamas family,yes am interested in obamas family
39,102,What is Fickle Creek Farm,Find general information about Fickle Creek Fa...,2,F0014,Find general information about Fickle Creek Fa...,Q00059,are you going to purchase anything there,i dont know yet i just want general info about...
84,105,Tell me about sonoma county medical services.,What medical services are available in Sonoma ...,2,F0025,What medical services are available in Sonoma ...,Q00457,are you interested in the human services depar...,no i am looking for doctors or hospitals in so...
120,108,Tell me about of Ralph Owen Brester.,Find biographical information about Ralph Owen...,1,F0037,Find biographical information about Ralph Owen...,Q00297,are you interested in learning more about ralp...,yes and his biography
159,109,I'm looking for information about mayo clinic ...,What medical services are available at the May...,2,F0040,What medical services are available at the May...,Q00256,are you interested in jobs at mayo clinic jack...,no im interested in services provided at mayo ...
...,...,...,...,...,...,...,...,...,...
9113,234,tell me about dark chocolate health benefits,What are the health benefits associated with e...,1,F0539,What are the health benefits associated with e...,Q00430,are you interested in the different candles da...,no im more interested in the health benefits o...
9127,238,Tell me bio of george bush sr.,Find biographies of US President George H.W. B...,2,F0552,Find biographies of US President George H.W. B...,Q00402,are you interested in the 1992 presidential el...,yes i would like to know about george bush srs...
9139,240,Tell me more about us presidents,Find a list of the full names of US presidents.,3,F0558,Find a list of the full names of US presidents.,Q00201,are you interested in finding out the middle n...,no i am interested in the names of all us pres...
9151,261,Tell me about folk remedies for a sore throat.,What folk remedies are there for soothing a so...,1,F0633,What folk remedies are there for soothing a so...,Q00208,are you interested in folk remedies from a spe...,no just any folk remedies for a sore throat


In [10]:
rdev = pd.read_table('./data/dev.tsv', sep = '\t', header = 0).drop_duplicates('topic_id')
rdev

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,101,Find me information about the Ritz Carlton Lak...,Find information about the Ritz Carlton resort...,2,F0010,Find information about the Ritz Carlton resort...,Q00697,are you looking for a specific web site,yes for the ritz carlton resort at lake las vegas
60,106,I'm looking for universal animal cuts reviews,Find testimonials of Universal Animal Cuts nut...,3,F0028,Find testimonials of Universal Animal Cuts nut...,Q01481,did universal animal cuts work for you,i need testimonials on the universal animal cu...
102,107,tell me about cass county missouri,Find demographic information about Cass County...,3,F0031,Find demographic information about Cass County...,Q00086,are you interested in a list of homes for sale...,no i want demographic info for cass county mo
192,114,Tell about an adobe indian house?,How does one build an adobe house?,2,F0063,How does one build an adobe house?,Q00057,are you going to purchase any specific product...,maybe
231,123,What is von Willebrand Disease?,What is von Willebrand Disease?,3,F0100,What is von Willebrand Disease?,Q00284,are you interested in learning about treatment...,id like to know what it is first
267,128,Tell me about atypical squamous cells,What do atypical squamous cells mean on a pap ...,2,F0114,What do atypical squamous cells mean on a pap ...,Q00081,are you interested in a desciption of aytypica...,yes and it must be related to pap smear tests
303,133,all men are created equal,"Who said \""all men are created equal\""?",2,F0134,"Who said ""all men are created equal""?",Q00796,are you looking for declaration of independenc...,i am looking for who said the qoute provided
378,139,Tell me more about Rocky Mountain News,discussion of the impending sale of the Rocky ...,2,F0156,discussion of the impending sale of the Rocky ...,Q00334,are you interested in news archives,no i am intrested in the sale of arocky mounta...
426,142,Find me information about the sales tax in Ill...,information about the sales tax in Illinois: w...,2,F0170,information about the sales tax in Illinois: w...,Q00241,are you interested in how the illinois state t...,no i would like to know the rate and what it i...
471,164,I'm looking for information on hobby stores,What hobby stores carry trains?,4,F0259,What hobby stores carry trains?,Q00659,are you looking for a radiocontrolled plane,no im looking for trains


In [11]:
df = requests.append(rdev)

In [12]:
df

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,1,Tell me about Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magazine photo essay ""Barack Oba...",Q00384,are you interested in seeing barack obamas family,yes am interested in obamas family
39,102,What is Fickle Creek Farm,Find general information about Fickle Creek Fa...,2,F0014,Find general information about Fickle Creek Fa...,Q00059,are you going to purchase anything there,i dont know yet i just want general info about...
84,105,Tell me about sonoma county medical services.,What medical services are available in Sonoma ...,2,F0025,What medical services are available in Sonoma ...,Q00457,are you interested in the human services depar...,no i am looking for doctors or hospitals in so...
120,108,Tell me about of Ralph Owen Brester.,Find biographical information about Ralph Owen...,1,F0037,Find biographical information about Ralph Owen...,Q00297,are you interested in learning more about ralp...,yes and his biography
159,109,I'm looking for information about mayo clinic ...,What medical services are available at the May...,2,F0040,What medical services are available at the May...,Q00256,are you interested in jobs at mayo clinic jack...,no im interested in services provided at mayo ...
...,...,...,...,...,...,...,...,...,...
2120,51,Tell me more about horse hooves,"Find information about horse hooves, their car...",2,F0858,Find information about horses' hooves and how ...,Q00025,are you asking about a horses feet,yes about horse hooves
2195,79,tell me about Voyager,Find information about the NASA Voyager spacec...,4,F0981,Find the homepage for the NASA Voyager mission.,Q00453,are you interested in the history of the voyager,no just take me to the homepage
2251,83,tell me about memory,Find information about human memory.,4,F0998,Find information about human memory.,Q00176,are you interested in computer memory,no
2293,256,Who is the patron saint of mental illness?,Who is the patron saint of mental illness?,1,F0609,Who is the patron saint of mental illness?,Q00304,are you interested in learning the back story ...,im interested in learning who it is


In [13]:
f = open('retrieved_docs_encoded.pkl', 'rb')

In [14]:
docs = {}
with yaspin().arc:
    while True:
        try:
            data = pickle.load(f)
            tid = list(data.keys())[0]
            docs[tid] = list(data.values())[0]
        except EOFError:
            break

0m 

In [15]:
ks = list(docs.keys())

In [16]:
tids = requests['topic_id'].to_numpy(dtype = int)

In [17]:
tids = np.intersect1d(tids, np.array(ks))

In [18]:
len(tids)

187

In [19]:
inputs = torch.zeros((len(tids), num_docs, embed_size))
cosines = torch.zeros((len(tids), num_docs, num_docs))
labels = torch.LongTensor(torch.zeros(len(tids)).long())
init_qrs = torch.zeros((len(tids), init_query_embed_size))

In [20]:
inputs.shape

torch.Size([187, 50, 768])

In [21]:
for i in tqdm(range(len(tids))):
    tid = tids[i]
    enc = docs[tid]['encoded_docs'][:50]
    enc = np.array(enc)
    csml = cosine_similarity(enc)
    inputs[i] = torch.Tensor(enc)
    A = torch.Tensor(csml)
    S = torch.sum(A, axis = 1)
    D = torch.diag(torch.sum(A, axis = 1))
    D_ = torch.Tensor(fractional_matrix_power(D, -0.5))
    A_ = torch.chain_matmul(D_, A, D_)
    cosines[i] = A_
    labels[i] = df.loc[df['topic_id'] == tid]['clarification_need'].to_numpy(dtype = np.int_)[0]
    init_qr = df.loc[df['topic_id'] == tid]['initial_request'].to_numpy(dtype = str)[0]
    init_qrs[i] = torch.Tensor(bert_model.encode(init_qr))

In [22]:
init_qrs.shape

torch.Size([187, 768])

In [23]:
labels = torch.LongTensor(labels.long())

In [24]:
labels

tensor([2, 3, 2, 3, 4, 2, 3, 2, 3, 2, 2, 3, 4, 1, 2, 2, 1, 2, 2, 2, 2, 3, 3, 2,
        2, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 2, 4, 2, 3, 2, 4, 3, 2, 4, 1, 2, 3, 1,
        4, 4, 2, 4, 3, 3, 1, 2, 4, 3, 4, 4, 3, 2, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3,
        2, 4, 2, 4, 4, 4, 4, 2, 3, 3, 2, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 3,
        2, 3, 2, 3, 3, 2, 3, 2, 2, 2, 4, 2, 2, 3, 3, 1, 2, 2, 4, 3, 2, 2, 2, 3,
        2, 2, 4, 4, 3, 3, 3, 3, 3, 2, 2, 3, 3, 2, 4, 2, 2, 3, 1, 3, 2, 3, 2, 2,
        3, 4, 3, 3, 1, 4, 3, 3, 1, 3, 2, 2, 1, 3, 2, 2, 2, 3, 2, 1, 1, 1, 2, 3,
        1, 2, 1, 1, 2, 1, 3, 4, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 3])

In [25]:
torch.max(torch.max(cosines[0]))

tensor(0.0312)

In [26]:
dev_tids = rdev['topic_id'].to_numpy(dtype = int)

In [27]:
dev_labels = rdev['clarification_need'].to_numpy(dtype = int)

In [28]:
dev_inputs = torch.zeros((len(dev_tids), num_docs, embed_size))
dev_cosines = torch.zeros((len(dev_tids), num_docs, num_docs))
dev_init_qrs = torch.zeros((len(dev_tids), init_query_embed_size))
dev_labels = torch.LongTensor(torch.zeros(len(dev_tids)).long())

In [29]:
for i in tqdm(range(len(dev_tids))):
    tid = dev_tids[i]
    enc = docs[tid]['encoded_docs'][:50]
    enc = np.array(enc)
    csml = cosine_similarity(enc)
    dev_inputs[i] = torch.Tensor(enc)
    A = torch.Tensor(csml)
    S = torch.sum(A, axis = 1)
    D = torch.diag(torch.sum(A, axis = 1))
    D_ = torch.Tensor(fractional_matrix_power(D, -0.5))
    A_ = torch.chain_matmul(D_, A, D_)
    dev_cosines[i] = A_
    dev_labels[i] = df.loc[df['topic_id'] == tid]['clarification_need'].to_numpy(dtype = np.int_)[0]
    init_qr = rdev.loc[rdev['topic_id'] == tid]['initial_request'].to_numpy(dtype = str)[0]
    dev_init_qrs[i] = torch.Tensor(bert_model.encode(init_qr))

In [30]:
dev_labels = torch.LongTensor(dev_labels.long())

In [31]:
dev_init_qrs.shape

torch.Size([50, 768])

In [32]:
train_dataset = torch.utils.data.TensorDataset(inputs, cosines, init_qrs, labels)
dev_dataset = torch.utils.data.TensorDataset(dev_inputs, dev_cosines, dev_init_qrs, dev_labels)
dataset = torch.utils.data.ConcatDataset([train_dataset, dev_dataset])
kfold = KFold(n_splits = k_folds, shuffle = True)

In [36]:
def train(epochs, batch_size):
    loss_fn = torch.nn.CrossEntropyLoss()
    results = {}
    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
        print(f"Fold: {fold}")
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, sampler = train_subsampler)
        test_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, sampler = test_subsampler)
        model = AmbiguityNetwork()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay = 0.1)
        curr_count = 0
        acc = 0
        curr_loss = 0
        for epoch in tqdm(range(epochs)):
            model.train()
            for idx, data in enumerate(train_loader):
                nf, A, iqs, tgts = data
                tgts = tgts - 1
                tgts = torch.LongTensor(tgts)
                preds = model(nf, A, iqs)
                loss = loss_fn(preds, tgts)
                m = torch.nn.Softmax()
                npreds = m(preds).detach().numpy()
                class_preds = np.argmax(npreds, axis = 1)
                acc += accuracy_score(tgts.detach().numpy(), class_preds)
                curr_loss += loss.item()
                curr_count += 1
                model.zero_grad()
                loss.backward()
                optimizer.step()
        curr_loss = curr_loss/curr_count
        acc = acc/curr_count

        with torch.set_grad_enabled(False):
            model.eval()
            val_acc = 0
            val_loss = 0
            val_prec = 0
            val_rec = 0
            val_f1 = 0
            total = 0
            count = 0
            for dev_idx, dev_data in enumerate(test_loader):
                nf, A, iqs, dev_labels = dev_data
                val_preds = model(nf, A, iqs)
                m = torch.nn.Softmax()
                val_npreds = m(val_preds).detach().numpy()
                class_preds = np.argmax(val_npreds, axis = 1)
                class_preds = class_preds + 1
                val_acc += accuracy_score(dev_labels, class_preds)
                val_loss += log_loss(dev_labels, val_npreds, labels = [1,2,3,4])
                val_prec += precision_score(dev_labels, class_preds, average = 'weighted')
                val_rec += recall_score(dev_labels, class_preds, average = 'weighted')
                val_f1 += f1_score(dev_labels, class_preds, average = 'weighted')
                count += 1
        results[fold] = {
            'training_loss': curr_loss,
            'training_acc': acc,
            'validation_loss': val_loss/count,
            'validation_acc': val_acc/count,
            'precision': val_prec/count,
            'recall': val_rec/count,
            'f1': val_f1/count
        }
        print(f"Loss: {curr_loss}, training acc: {acc}, validation_loss: {val_loss/count}, validation acc: {val_acc/count}")

    train_acc = 0
    train_loss = 0
    val_acc = 0
    val_loss = 0
    prec = 0
    f1 = 0
    rec = 0
    for key, value in results.items():
        train_acc += value['training_acc']
        train_loss += value['training_loss']
        val_acc += value['validation_acc']
        val_loss += value['validation_loss']
        prec += value['precision']
        rec += value['recall']
        f1 += value['f1']
    print('Final Metrics:')
    print(f"Training loss: {train_loss/k_folds}, Training acc: {train_acc/k_folds}, Validation loss: {val_loss/k_folds}, Validation acc: {val_acc/k_folds}, Precision: {prec/k_folds}, Recall: {rec/k_folds}, F1: {f1/k_folds}")

In [38]:
train(100, batch_size)

Fold: 0


<ipython-input-36-867ad9dd180e>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  npreds = m(preds).detach().numpy()



Loss: 0.6931842352371467, training acc: 0.7921520467836296, validation_loss: 1.5840927896276118, validation acc: 0.29
Fold: 1


<ipython-input-36-867ad9dd180e>:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_npreds = m(val_preds).detach().numpy()
D:\Installations\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-36-867ad9dd180e>:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_npreds = m(val_preds).detach().numpy()
D:\Installations\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

<ipython-input-36-867ad9dd180e>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  npreds = m(preds).detach().numpy()


...


Loss: 0.7244401293992996, training acc: 0.7764736842105293, validation_loss: 1.395842742409025, validation acc: 0.3885714285714286
Fold: 4


<ipython-input-36-867ad9dd180e>:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_npreds = m(val_preds).detach().numpy()
D:\Installations\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-36-867ad9dd180e>:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_npreds = m(val_preds).detach().numpy()
<ipython-input-36-867ad9dd180e>:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_npreds = m(val_preds).detach().numpy()
D:\Installations\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedM

<ipython-input-36-867ad9dd180e>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  npreds = m(preds).detach().numpy()



Loss: 0.713829603759866, training acc: 0.7905789473684234, validation_loss: 1.4415476806036065, validation acc: 0.30857142857142855
Final Metrics:
Training loss: 0.7133861146098689, Training acc: 0.7849859649122837, Validation loss: 1.441109368102891, Validation acc: 0.35357142857142854, Precision: 0.35049047619047624, Recall: 0.35357142857142854, F1: 0.32756310832025115


<ipython-input-36-867ad9dd180e>:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_npreds = m(val_preds).detach().numpy()
D:\Installations\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-36-867ad9dd180e>:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_npreds = m(val_preds).detach().numpy()
D:\Installations\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

In [ ]:
model = AmbiguityNetwork()
model.load_state_dict(torch.load('saved_model_graphcnn.pt'))
model.eval()

In [ ]:
val_preds = model(dev_inputs, dev_cosines, dev_init_qrs)
m = torch.nn.Softmax()
val_npreds = m(val_preds).detach().numpy()
class_preds = np.argmax(val_npreds, axis = 1)
class_preds = class_preds + 1
prec_score = precision_score(dev_labels, class_preds, average = 'weighted')
rec_score = recall_score(dev_labels, class_preds, average = 'weighted')
f1 = f1_score(dev_labels, class_preds, average = 'weighted')
print(prec_score, rec_score, f1)

In [ ]:
dev_inputs[0]

In [ ]:
preds = model(dev_inputs, dev_cosines)

In [ ]:
m = torch.nn.Softmax()

preds = m(preds).detach().numpy()

In [ ]:
preds

In [ ]:
class_preds = np.argmax(preds, axis = 1)
class_preds

In [ ]:
len(class_preds)

In [ ]:
class_preds = class_preds + 1

In [ ]:
class_preds

In [ ]:
outs = [(dev_tids[i], class_preds[i]) for i in range(len(preds))]
outs = np.array(outs)
outs

In [ ]:
np.savetxt('preds_dev_doc2vec_cosine_10gcn_10gcn.txt', outs, fmt="%s %s")

In [ ]:
!python clariq_eval_tool.py --eval_task clarification_need \
                                 --data_dir ./data/ \
                                 --experiment_type dev \
                                 --run_file preds_dev_doc2vec_cosine_10gcn_10gcn.txt \
                                 --out_file out